In [31]:
# From the website https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils

import glob 
from pathlib import Path

In [32]:
class CustomDatasetByFolder(Dataset):

    def __init__(self, root_dir, transform=None):
        # super.__init__(self,CustomDatasetByFolder)

        self.root_dir = root_dir
        self.transform = transform

        exts = [".jpg", ".jpeg", ".tiff", '.png']
        mainpath = f'{self.root_dir}'
        self.filenames = [p for p in Path(mainpath).rglob('*') if p.suffix in exts]
        
        # search_path = f'{self.root_dir}/**/*.png'
        # self.filenames =  glob.glob(search_path, recursive = True)

    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, idx):
        
        filename = self.filenames[idx]
        image = io.imread(filename)

        if str(filename).find('cats') != -1:
            label = 0
        else:
            label = 1

        # return {'image': image, 'label': label}
        # Convert to having RGB as the first dimension as feature vector
        # image = np.transpose(image,(2,0,1))

        if self.transform:
            image = self.transform(image)

        return image, label


In [33]:
class CustomDatasetFromCSV(Dataset):

    def __init__(self, dataset_fn, transform=None):
        # super.__init__(self,CustomDatasetByFolder)

        self.dataset_fn = dataset_fn
        self.transform = transform

        self.df = pd.read_csv(self.dataset_fn)

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        filename = self.df.iloc[idx]['filename']
        # filename = self.filenames[idx]
        image = io.imread(filename)

        label = self.df.iloc[idx]['category']        

        # Convert to having RGB as the first dimension as feature vector
        # image = np.transpose(image,(2,0,1))

        if self.transform:
            image = self.transform(image)

        return image, label

In [40]:
transform_list = transforms.Compose(
    [transforms.ToTensor(),
    #  transforms.Resize((64,64)),
     transforms.RandomResizedCrop(size=(64,64)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


folderDataset = CustomDatasetByFolder('./example_dataset', transform=transform_list)
print(len(folderDataset))
folderDataset[-1]

folderLoader = torch.utils.data.DataLoader(folderDataset, batch_size=10,
                                          shuffle=True)
dataiter = iter(folderLoader)
images, labels = next(dataiter)
print(images.shape)


294
torch.Size([10, 3, 64, 64])


In [23]:
csvDataset = CustomDatasetFromCSV('./cat_dog_dataset.csv')
print(len(csvDataset))
csvDataset[0]

csvLoader = torch.utils.data.DataLoader(csvDataset, batch_size=2,
                                          shuffle=True)
dataiter = iter(folderLoader)
images, labels = next(dataiter)


294


RuntimeError: stack expects each tensor to be equal size, but got [667, 1000, 3] at entry 0 and [100, 100, 3] at entry 1